In [1]:
import bandit
from numpy.random import uniform
from numpy.linalg import solve
import numpy as np
import state_params
import random
from scipy.stats import norm
import TS
from scipy.linalg import block_diag

In [2]:
###initializing variables

In [3]:
def get_vec_indices(pZ):
    return [i for i in range(2,pZ+1)]

In [4]:
sp = state_params.state_params(10)

In [5]:
def eta(x):
    return 1

In [6]:
pZ = 2
pi_max = .8
pi_min = .1


sigma = 1


xi  = 10 
eta_init = 0
gamma_mdp = .9
lambda_knot = .9 
prob_sedentary = .9 
weight = .5 
z_init = get_vec_indices(pZ)
x_index = 2+pZ
avail_index = 3+pZ
prob_index = 5+pZ
rwrd_index = 6+pZ


In [5]:
def gen_nextdosage(x,a):
    anti_sed = int(uniform() < 0.3)
    event = (anti_sed+a)>0
    x_next = 0.9*x + event
    
    return x_next

In [6]:
def prob_cal_ts(z,x,mu,Sigma,init):
    pos_mean = np.dot(bandit.feat2_function(z,x),mu)
    pos_var = np.dot(np.dot(np.transpose(bandit.feat2_function(z,x)),Sigma),bandit.feat2_function(z,x))
    pos_var = max(0,pos_var)

  
 
  
    # probability
    pit_zero = norm.cdf((pos_mean)/(pos_var**.5))
  
    # clipping
    prob =  min(bandit.py_c_func(init.pi_max, max(bandit.py_c_func(init.pi_min, pit_zero))))
  
    return prob


In [7]:
def make_batch(t,Z,X,I,A,prob,R):
    temp = [t]
    temp.extend(Z)
    temp.extend([X,I,A,prob,R])
    return temp

In [29]:
def get_xz_matrix(batch,init):
    new_matrix = [[b[init.x_index]]+b[init.z_index[0]:init.z_index[-1]+1] for b in batch if b[init.avail_index]==1]
    return new_matrix
    
    
    

In [24]:
def transform_f1(xz):
    return [[1,row[2],row[0],row[1]] for row in xz]
    

In [10]:
def transform_f2(xz):
    return [[1,row[-1],row[0]] for row in xz]

In [11]:
def get_actions(batch,init):
    return [b[init.action_index] for b in batch if b[init.avail_index]==1]

In [32]:
xz= get_xz_matrix(b,sp)

f1 = transform_f1(xz)

In [19]:
gr  =get_actions(b,sp)

In [55]:
def get_X_trn(F1,actions,F2):
    to_return = []
    for i in range(len(F2)):
        a = np.multiply(actions[i],F2[i])
        row = np.concatenate((np.array(F1[i]),temp[1]))
        to_return.append(row)
    return to_return
    

In [53]:
temp= [np.multiply(gr[i],f1[i]) for i in range(len(f1))]
np.concatenate((np.array([0,0,0]),temp[1]))

array([ 0.        ,  0.        ,  0.        ,  1.        ,  0.49336811,
        0.        ,  0.45777434])

In [12]:
def get_probs(batch,init):
    return [b[init.prob_index] for b in batch if b[init.avail_index]==1]

In [96]:
 xz = matrix(batch[index, c(input$x.index, input$z.index)], nrow = sum(index))
      action <- batch[index, input$action.index]
      prob <- batch[index, input$prob.index]
      
      # forming the prior 
      mu.tmp <- c(input$mu1, input$mu2)
      Sigma.tmp <- as.matrix(bdiag(input$Sigma1, input$Sigma2))
      
      # feature matrix
      F1 <- t(apply(xz, 1, function(s) input$feat1(x = s[1], z = s[-1])))
      F2 <- t(apply(xz, 1, function(s) input$feat2(x = s[1], z = s[-1])))
      
      # calculate posterior (batch update)
      X.trn <- cbind(F1, action * F2)
      Y.trn <- batch[index, input$rwrd.index]
      temp <- post.cal(X.trn, Y.trn, input$sigma, mu.tmp, Sigma.tmp)
      
      # return the post dist of txt eff
      txt.index <- tail(1:ncol(X.trn), ncol(F2)) # interaction terms
      list(mean = temp$mean[txt.index], var = temp$var[txt.index, txt.index])

array([[ 0.1,  0. ,  0. ],
       [ 0. ,  0.1,  0. ],
       [ 0. ,  0. ,  0.1]])

In [13]:
#what does this stand for? change the name
def txt_effect_update(batch,init):
    avail = [b[init.avail_index] for b in batch]
    #avail <- batch[, input$avail.index]
    
    ##how can this ever equal 1?
    index = [int(a==1) for a in avail]
    
    if sum(index)==1:
        
        return [init.mu_2,init.sigma_2]
    else:
        #what is this line doing?
        xz = get_xz_matrix(batch,init)
        #action <- batch[index, input$action.index]
        
        mu_tmp = init.mu_1+ init.mu_2
        Sigma_tmp = block_diag(sp.sigma_1,sp.sigma_2)
        
        actions = get_actions(batch,init)
        probs = get_probs(batch,init)
        
        f_one = transform_f1(xz)
        f_two = transform_f2(xz)
        
        X_trn = get_X_trn(f_one,actions,f_two)
        

In [14]:
def post_cal(X, Y, sigma, mu, Sigma):
    
    inv_Sigma = solve(Sigma,np.eye(len(Sigma[0])))
    term_one = np.dot(np.transpose(X),X)+sigma**2*inv_Sigma
    term_two = np.dot(np.transpose(X),Y)+np.dot(sigma**2*inv_Sigma,mu)
    pos_mean = solve(term_one,term_two)
    
    
    #pos.mean <- c(solve(t(X)%*%X + sigma^2*inv.Sigma, t(X)%*%Y + sigma^2*inv.Sigma %*% mu));
    #pos.var <- sigma^2 * solve(t(X) %*% X +  sigma^2 * inv.Sigma);
    
    #list(mean = pos.mean, var = pos.var)
  

    #check then keep going

In [15]:
def ts_policy_updates(batch, init):
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-15-d80134e463ae>, line 3)

In [16]:
def simulate(pZ,init):
    total_reward = 0
  # simulation
    nT = 450

      # initial state
    Z_init = [uniform() for i in range(pZ)]
    X_init =0
    I_init = (uniform() < 0.8)
  

  
    # policy initilization
    mu_beta = init.mu_2
    Sigma_beta = init.sigma_2
  
    # placeholder
    batch = [[] for i in range(nT)]
  
  
    # performance matrix

  
  
    for t in range(nT):
    
    
    # current state 
    
        if(t==0):
      
            Z_next = Z_init
            X_next = X_init
            I_next = I_init
      
    
    
        Z = Z_next
        X = X_next
        I = I_next
    
    # action selection
    
        if I == 1:
      
          # calculate prob
      
          prob = TS.prob_cal_ts(Z, X, mu_beta, Sigma_beta, init)
      
          # sample the action
          A = int(uniform() < prob)
      
        else:
      
          prob = 0
          A = 0
      
        
    
        R = 1+Z[0]+(Z[0]**2)-(.05*X)+A*(1-.1*X)+np.random.normal(scale=1)
        Z_next = [uniform() for i in range(pZ)]
        X_next = TS.gen_nextdosage(X,A)
        I_next = (uniform()<.8)
        
        
        batch[t]=TS.make_batch(t,Z,X,I,A,prob,R)
        
        
        total_reward = total_reward+R

    
        #policy_update_ts(batch, init)
    print(total_reward)
    return batch,total_reward
  
    


In [17]:
b,t =simulate(sp.pZ,sp)

814.3561937412969


In [81]:
zs =[]
for v in b:
    zs.append(v[5])
np.array(zs).mean()

0.39555555555555555

In [ ]:
    # Collect Reward and State Transition
        R <-  1 + Z[1] + Z[1]^2 - 0.05 * X + A * (1 - 0.1*X) + rnorm(1, sd = 1)
        Z.next <- runif(pZ)
        X.next <- gen_nextdosage(X, A)
        I.next <- (runif(1) < 0.8)
    
    
    # add to the batch set
        batch <- rbind(batch, c(t, Z, X, I, A, prob, R))
    
    
    # performance
        tot.rwrd <- tot.rwrd + R
    
    # nightly update
    
        if(t%5 == 0){
      
      # update the txt effect posterior
          temp = policy.update.ts(batch, input)
          mu.beta = temp$mean
          Sigma.beta = temp$var
      
      
          #print(t%/%5)
      
        }

In [7]:
#z, x, mu, Sigma, eta, input
bandit.calculate_prob(z,0,[1,0,0],sigma_2,0,0)

NameError: name 'z' is not defined

In [5]:
[random.random() for i in range(2)]

[0.3730834968521465, 0.7865302742162711]

In [8]:
0+(.4>0)

1

In [21]:
i = [[] for i in range(50)]

In [22]:
i[0]=[12,3,3]

In [23]:
i

[[12, 3, 3],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [56]:
sp.pi_max

0.8